In [1]:
import json
import pickle
from pathlib import Path

import spacy
import random
from tqdm import tqdm

In [2]:
with open('Corona2.json') as cs:
    data = json.load(cs)


In [3]:
### Sample Data
### [
###  ('Content1',{'entities':[(7,19,'PERSON),(21,28,'LOC)]}),
###  ('Content2',{'entities':[(7,19,'PERSON),(21,28,'LOC)]}),
### ]

In [17]:
spacy_data=[]
for element in data['examples']:
    ent_dict={}
    ent_list=[]
    index=[]
    for ent in element['annotations']:
        if ent['human_annotations']!=[] and ent['start'] not in index and ent['end'] not in index:
            index.append(ent['start'])
            index.append(ent['end'])
            ent_list.append((ent['start'],ent['end'],ent['tag_name']))
    ent_dict['entities']=ent_list
    spacy_data.append((element['content'],ent_dict))

In [18]:
spacy_data

[("While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]",
  {'entities': [(360, 371, 'Medicine'),
    (383, 408, 'Medicine'),
    (104, 112, 'MedicalCondition'),
 

In [6]:
with open('spacy_data.pkl', 'wb') as file:
      
    # A new file will be created
    pickle.dump(spacy_data, file)

In [19]:
model = None
output_dir=Path("model")
n_iter=100

In [20]:
#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


In [21]:
from spacy.training.example import Example

In [22]:

for _, annotations in spacy_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(spacy_data)
        losses = {}
        for text, annotations in tqdm(spacy_data):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

  0%|          | 0/31 [00:00<?, ?it/s]c:\Users\rajee\Python_projects\Oher ML projects\Medical_NER\med_ner\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Influenza, commonly known as "the flu", is an infe..." with entities "[(191, 202, 'MedicalCondition'), (0, 9, 'MedicalCo...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
  6%|▋         | 2/31 [00:00<00:01, 15.60it/s]c:\Users\rajee\Python_projects\Oher ML projects\Medical_NER\med_ner\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "While bismuth compounds (Pepto-Bismol) decreased t..." with entities "[(360, 371, 'Medicine'), (383, 408, 'Medicine'), (...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities 

{'ner': 1487.6876494548826}


100%|██████████| 31/31 [00:01<00:00, 17.87it/s]


{'ner': 409.6899568421771}


100%|██████████| 31/31 [00:01<00:00, 17.79it/s]


...

{'ner': 42.84927176066932}


100%|██████████| 31/31 [00:02<00:00, 12.09it/s]


{'ner': 49.95885315503507}


100%|██████████| 31/31 [00:01<00:00, 16.75it/s]

{'ner': 42.330493922585774}


In [23]:

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to model


In [24]:
for text, _ in spacy_data:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('amantadine', 'Medicine'), ('rimantadine', 'Medicine'), ('influenza', 'MedicalCondition'), ('M2 inhibitors', 'Medicine')]
Entities [('bedaquiline', 'Medicine'), ('TB therapy', 'Medicine'), ('bedaquiline', 'Medicine')]
Entities [('severe acute respiratory syndrome', 'MedicalCondition'), ('SARS', 'MedicalCondition'), ('coronavirus', 'Pathogen'), ('SARS', 'MedicalCondition'), ('SARS coronavirus', 'Pathogen'), ('SARS-CoV', 'Pathogen')]
Entities [('bacteria', 'Pathogen'), ('pathogenic bacteria', 'Pathogen'), ('Streptococcus', 'Pathogen'), ('Pseudomonas', 'Pathogen'), ('foodborne illnesses', 'MedicalCondition'), ('Shigella', 'Pathogen'), ('Campylobacter', 'Pathogen'), ('Salmonella', 'Pathogen'), ('tetanus', 'MedicalCondition'), ('typhoid fever', 'MedicalCondition'), ('diphtheria', 'MedicalCondition'), ('syphilis', 'MedicalCondition'), ("Hansen's disease", 'MedicalCondition')]
Entities [('herpes simplex virus', 'Pathogen'), ('Yersinia pestis.[155', 'Pathogen')]
Entities [('bismuth 

#### Load trained model

In [26]:
nlp1 = spacy.load(r"model") #load the best model
     

In [28]:
doc = nlp1("Mrs. Smith was prescribed 500mg of amoxicillin three times a day for her bacterial infection.") # input sample text

print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('amoxicillin', 'Medicine')]


In [30]:
text = str(input("Enter the text to extract the entities: "))

In [31]:
text

'ghhaf akdhf'